In [6]:
%matplotlib inline
from tiptop import *

from tiptopGUI import *
from configparser import ConfigParser
from configparser import RawConfigParser
from ipyfilechooser import FileChooser
from functools import partial

config = ConfigParser()
config.optionxform = str

basicPath = '/home/frossi/dev/TIPTOP/localTest'
inititalFilename = 'params.ini'

fileSelector = FileChooser(basicPath)
fileSelector.default_path = basicPath
fileSelector.default_filename = inititalFilename
fileSelector.reset()
fileSelector.reset(path=basicPath, filename=inititalFilename)
fileSelector.show_hidden = False
fileSelector.use_dir_icons = True
fileSelector.show_only_dirs = False
fileSelector.filter_pattern = '*.ini'
fileSelector.title = '<b>Select the configuration file to import</b>'

output = widgets.Output()

tab = widgets.Tab()

computeButton = widgets.Button(description="Compute")
saveButton = widgets.Button(description="Save to config file")
outputMessage = widgets.Label(value="Compute Done!")
variablesToWidgets = {}
widgetsToVariables = {}
variablesToListWidgets = {}
sections = []
sectionsVariables = {}
variablesValues = {}

def updateGlobals(variablesToWidgets):
    global variablesValues
    variablesValuesNew = {}
    for key in variablesToWidgets:
        w = variablesToWidgets[key]
        if type(w) is not widgets.HBox:
            variablesValuesNew[key] = w.value
        else:
            for aw in w.children:
                variablesValuesNew[key] = []
                if type(aw) is not widgets.HBox:
                    variablesValuesNew[key] = []
                else:
                    variablesValuesNew[key] = []
                    for aaw in aw.children:
                        variablesValuesNew[key].append(aaw.value)
    variablesValues = variablesValuesNew


def saveToParamsFile(filename):
    global sections, sectionsVariables, variablesValues
    
    updateGlobals(variablesToWidgets)

    configW = RawConfigParser()
    configW.optionxform = lambda option: option
    for section in sections:
        configW[section] = {}
        for key in sectionsVariables[section]:
            if type(variablesValues[key])==list and len(variablesValues[key])==0:
                configW[section][key] = "[]"
            elif variablesValues[key] is None:
                configW[section][key] = "None"
            else:
                configW[section][key] = str(variablesValues[key])

    with open(filename+'.ini', 'w') as configfile:
        configW.write(configfile)


def on_compute_button_clicked(b):
    outputResult = widgets.Output()
    tab.children = (*tab.children, (outputResult))        
    ii = len(tab.children)-1
    tab.set_title(ii, "Result")
    
    fullInputPath = '' # fileSelector.selected_path +'/'

    # parametersFile = fileSelector.selected_filename[:-4]
    
    parametersFile = 'tmpParams'
    saveToParamsFile(parametersFile)
    
    baseOutpuPath = fullInputPath + 'output/'
    outputFile = "test"    
    
    with outputResult:
        overallSimulation(fullInputPath, parametersFile, baseOutpuPath, outputFile, doConvolve=True, pitchScaling = 0.9)    
    display(tab, outputResult)
     

def on_save_button_clicked(b):
    return

def on_listItemsCount_changed(b):    
    return

def initTab():
    titles = []
    children = []
    titles.append("Commands")
    children.append(widgets.VBox([fileSelector, saveButton, computeButton]))        
    tab.children = children
    for ii,tt in enumerate(titles):
        tab.set_title(ii,tt)    
    display(tab, output)
    
def buildStructureFromFile(filename):
    sectionsVariables = {}
    variablesValues = {}
    sections = []
    aa = config.read(filename)    
    for section in config.sections():
        sectionsVariables[section] = []
        for key in config[section]:
            variablesValues[key] = eval(config[section][key])
            if variablesValues[key] is not None:
                sectionsVariables[section].append(key)

    sections = [str(section) for section in config.sections()]    
    return sections, sectionsVariables, variablesValues
    
def newBuildGuiFromFile(chooser):
    global variablesToWidgets
    global widgetsToVariables    
    global sections, sectionsVariables, variablesValues
    
    label_layout = Layout(width='330px',height='30px')
    style = {'description_width': '240px'}    
    label_layout_short = Layout(width='90px',height='30px')
    style_short = {'description_width': '10px'}
    children = []
    titles = []    
    titles.append("Commands")
    children.append(widgets.VBox([chooser, saveButton, computeButton]))
    sections, sectionsVariables, variablesValues = buildStructureFromFile(chooser.selected)

    for section in sections:
        w_list = []
        for key in sectionsVariables[section]:
            vv = variablesValues[key]            
            ktext = variables_text[key]
            _min, _max, _step, _t = variables_min_max_type[key]
            if ktext=="":
                ktext = key
            if type(vv)==list:
                listLenght = len(vv)
                l_list = []
                a_list = []
                widgetsCounter = widgets.BoundedIntText(value=listLenght, min=1, max=30, step=1, description=ktext, disabled=False, layout=label_layout, style=style)
                l_list.append(widgetsCounter)
                
                for vi in vv:
                    if _t==float:
                        newListWidget = widgets.BoundedFloatText(value=vi, min=_min, max=_max, step=_step, description="_", disabled=False, layout=label_layout_short, style=style_short)
                    elif _t==int:
                        newListWidget = widgets.BoundedIntText(value=vi, min=_min, max=_max, step=_step, description="_", disabled=False, layout=label_layout_short, style=style_short)

                    a_list.append(newListWidget)
 
                newWidget1 = widgets.HBox( l_list, layout=Layout(height='auto', flex_flow='row wrap') )
                newWidget2 = widgets.HBox( a_list, layout=Layout(height='auto', width='900px', flex_flow='row wrap') )
                newWidget = widgets.HBox( [newWidget1, newWidget2], layout=Layout(height='auto') )
            
                variablesToListWidgets[key] = newWidget2
                display(newWidget, output)

                def on_nElementsChange(change, akey):
                    dd = int(change.new)-int(change.old)
                    if dd > 0:
                        ww = widgets.BoundedFloatText(value=_min, min=_min, max=_max, step=_step, description="_", disabled=False, layout=label_layout_short, style=style_short)
                        variablesToListWidgets[akey].children=tuple(list(variablesToListWidgets[akey].children) + [ww]) 
                    elif dd < 0:
                        aa = variablesToListWidgets[akey].children[-1]
                        variablesToListWidgets[akey].children=tuple(list(variablesToListWidgets[akey].children[:-1])) 
#                        close(aa)
                        
                widgetsCounter.observe(partial(on_nElementsChange, akey=key), 'value')
            elif _t==float:
                newWidget = widgets.BoundedFloatText(value=vv, min=_min, max=_max, step=_step, description=ktext, disabled=False, layout=label_layout, style=style)
            elif _t==int:
                newWidget = widgets.BoundedIntText(value=vv, min=_min, max=_max, step=_step, description=ktext, disabled=False, layout=label_layout, style=style)

            w_list.append(newWidget)            
            variablesToWidgets[key] = newWidget
            
            
        children.append(widgets.VBox(w_list, layout=Layout(overflow_x='auto', overflow_y='auto')))
    titles1 = [str(sections_text[section]) for section in sections]
    titles += titles1    
    tab.children = children
    for ii,tt in enumerate(titles):
        tab.set_title(ii,tt)        
    display(tab, output)

def printAllGui(variablesToWidgets):
    for key in variablesToWidgets:
        w = variablesToWidgets[key]
        if type(w) is not widgets.HBox:
            print(key, w.value)
        else:
            for aw in w.children:
                if type(aw) is not widgets.HBox:
                    print(key, aw.value)                    
                else:
                    for aaw in aw.children:
                        print(key, aaw.value)                    
                    

computeButton.on_click(on_compute_button_clicked)
saveButton.on_click(on_save_button_clicked)
fileSelector.register_callback(newBuildGuiFromFile)

initTab()

Output()